In [1]:
%matplotlib inline
%precision 16
import numpy
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg
from scipy.integrate import quad

In [2]:
K = 1200.0
T = 0.125
t = 0.0
r = 0.03
kappa = 2
theta = 0.05
sigma = 0.01
rho = -0.5
_lambda = 0.0

In [3]:
#def Heston_Integral(f, x)
   # return
def Heston_Integral(phi, K, S, v, T, t, r, kappa, theta, sigma, rho, _lambda, j):
    
    u_1 = 1.0 / 2.0
    u_2 = - 1.0 / 2.0
    a = kappa * theta
    rspi = rho * sigma * phi * 1j
    tau = T - t
    
    b_1 = kappa + _lambda - rho * sigma
    b_2 = kappa + _lambda
    
    d_1 = numpy.sqrt((rspi - b_1)**2 - sigma**2 * (2.0 * u_1 * phi * 1j - phi**2))
    d_2 = numpy.sqrt((rho * sigma * phi * 1j - b_2)**2 - sigma**2 * (2.0 * u_2 * phi * 1j - phi**2))
    
    g_1 = (b_1 - rspi + d_1) / (b_1 - rspi - d_1)
    g_2 = (b_2 - rspi + d_2) / (b_2 - rspi - d_2)
   
    D_1 = (b_1 - rspi + d_1) / sigma**2 * ((1.0 - numpy.exp(d_1 * tau)) / (1.0 - g_1 * numpy.exp(d_1 * r)))
    D_2 = (b_2 - rspi + d_2) / sigma**2 * ((1.0 - numpy.exp(d_2 * tau)) / (1.0 - g_2 * numpy.exp(d_2 * r)))
    
    C_1 = r * phi * 1j * tau + a / sigma**2 * ((b_1 - rspi + d_1) * tau - 2.0 * numpy.log((1.0 - g_1 * numpy.exp(d_1 * tau))/(1.0 - g_1)))
    C_2 = r * phi * 1j * tau + a / sigma**2 * ((b_2 - rspi + d_2) * tau - 2.0 * numpy.log((1.0 - g_2 * numpy.exp(d_2 * tau))/(1.0 - g_2)))   
    
    f_1 = numpy.exp(C_1 + D_1 * v + 1j * phi * numpy.log(S))
    f_2 = numpy.exp(C_2 + D_2 * v + 1j * phi * numpy.log(S))
    
    if(j==1):
        return numpy.real((numpy.exp(-1j * phi * numpy.log(K)) * f_1)/(1j * phi))
    elif(j==2):
        return numpy.real((numpy.exp(-1j * phi * numpy.log(K)) * f_2)/(1j * phi))

In [4]:
# The closed-form solution for Heston PDE
def Exact_Solution(S, v): 
    P_1 = 0.5 + 1.0 / numpy.pi * \
          quad(Heston_Integral, 0, 1000, args=(K, S, v, T, t, r, kappa, theta, sigma, rho, _lambda, 1))[0]
    P_2 = 0.5 + 1.0 / numpy.pi * \
          quad(Heston_Integral, 0, 1000, args=(K, S, v, T, t, r, kappa, theta, sigma, rho, _lambda, 2))[0]
    
    Call = S * P_1 - K * numpy.exp(- r * (T - t)) * P_2
    return Call

In [9]:
print Exact_Solution(0, 0)

nan


/Users/yuhsuan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: RuntimeWarning: divide by zero encountered in log
/Users/yuhsuan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:27: RuntimeWarning: divide by zero encountered in log
/Users/yuhsuan/anaconda/lib/python2.7/site-packages/scipy/integrate/quadpack.py:352: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)
